In [1]:
import tensorflow as tf
import numpy as np
from tensorflow import keras
from tensorflow.keras.layers import InputLayer


(X_train, y_train), (X_test, y_test) = tf.keras.datasets.boston_housing.load_data()


In [2]:
import time
import os

root_logdir = os.path.join(os.curdir, "tb_logs")


def get_run_logdir(parName,parValue): 
    run_id = str(time.time()) + "_" + str(parName) + "_" + str(parValue)
    return os.path.join(root_logdir, run_id)

#run_logdir = get_run_logdir()
#tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

In [3]:
def build_model(n_hidden, n_neurons, optimizer, learning_rate, momentum=0):   # regression
    
    
    
    
    model = tf.keras.models.Sequential()
    
    # input layer
    
    model.add(InputLayer(input_shape=(X_train.shape[1], ), name='Input_Layer'))

    
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
        
        
    model.add(keras.layers.Dense(1))
    
    optimizer.learning_rate = learning_rate
    
    if momentum != 0:
        optimizer.momentum = momentum
        
    #optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    
    model.compile(loss="mse", optimizer=optimizer,metrics=["mae"])
    
    return model



In [4]:
# we clean session before experiments

tf.keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)


def cleanSession():
    # we clean session before experiments

    tf.keras.backend.clear_session()
    np.random.seed(42)
    tf.random.set_seed(42)

In [5]:
import pickle


early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,verbose=1,min_delta=1)

defaultNOL = 1
defaultLR = 0.00001
defaultNeuronsOnLayer = 25
optimizer = keras.optimizers.SGD()


In [6]:
# we do a trail experiment

run_logdir = get_run_logdir("NOL",defaultNOL)
tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

model = build_model(defaultNOL,defaultNeuronsOnLayer,optimizer,defaultLR)

history = model.fit(X_train, y_train, epochs=100,validation_split = 0.1, callbacks=[early_stopping_cb,tensorboard_cb])

Epoch 1/100
12/12 [==============================] - 0s 13ms/step - loss: 117101.7812 - mae: 123.7935 - val_loss: 108.1399 - val_mae: 8.5337
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 163.8253 - mae: 9.5988 - val_loss: 68.6999 - val_mae: 6.7683
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 113.2871 - mae: 7.8936 - val_loss: 49.6265 - val_mae: 5.5180
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 97.5250 - mae: 7.2386 - val_loss: 44.5197 - val_mae: 5.2958
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 91.5391 - mae: 7.0076 - val_loss: 42.7563 - val_mae: 5.2981
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 87.4378 - mae: 7.0010 - val_loss: 42.5495 - val_mae: 5.0830
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 86.9827 - mae: 6.7954 - val_loss: 41.1674 - val_mae: 5.1446
Epoch 8/100
12/12 [==============================] - 0s 2ms

In [7]:
# first experiment

# we do a trail experiment

firstExpResults = []
learningRates = [10**-6,10**-5,10**-4]

for lr in learningRates:

    print("-------------------------------------------------")
    
    run_logdir = get_run_logdir("lr",lr)
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

    model = build_model(defaultNOL,defaultNeuronsOnLayer,optimizer,lr)

    history = model.fit(X_train, y_train, epochs=100,validation_split = 0.1, callbacks=[early_stopping_cb,tensorboard_cb])
    history.history['loss'].sort()
    
    lossHis = history.history["loss"][0]
    print(str(lossHis) + " : loss history ----")
    
    history.history['mae'].sort()
    maeHis = history.history['mae'][0]
    print(str(maeHis) + " : mae history ----")

    firstExpResults.append((lr,lossHis,maeHis))
    
    print("-------------------------------------------------")


    

pickle_out = open("lr.pickle","wb")
pickle.dump(firstExpResults, pickle_out)
pickle_out.close()

cleanSession()

-------------------------------------------------
Epoch 1/100
12/12 [==============================] - 0s 13ms/step - loss: 4096.3271 - mae: 52.9450 - val_loss: 518.8672 - val_mae: 19.9679
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 383.5276 - mae: 16.5246 - val_loss: 252.1320 - val_mae: 13.7516
Epoch 3/100
12/12 [==============================] - 0s 3ms/step - loss: 292.2751 - mae: 14.2545 - val_loss: 207.5724 - val_mae: 12.1838
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 269.8958 - mae: 13.5324 - val_loss: 189.7364 - val_mae: 11.4592
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 250.9415 - mae: 12.9621 - val_loss: 175.8954 - val_mae: 10.8495
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 233.2527 - mae: 12.3469 - val_loss: 165.9992 - val_mae: 10.4661
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 216.2453 - mae: 11.7595 - val_loss: 148.8138 - val_ma

In [8]:
# second experiment

# we do a trail experiment

ExpResults = []

for hiddenLayers in range (0,4):

    print("-------------------------------------------------")
    
    run_logdir = get_run_logdir("hl",hiddenLayers)
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

    model = build_model(hiddenLayers,defaultNeuronsOnLayer,optimizer,defaultLR)

    history = model.fit(X_train, y_train, epochs=100,validation_split = 0.1, callbacks=[early_stopping_cb,tensorboard_cb])
    history.history['loss'].sort()
    
    lossHis = history.history["loss"][0]
    print(str(lossHis) + " : loss history ----")
    
    history.history['mae'].sort()
    maeHis = history.history['mae'][0]
    print(str(maeHis) + " : mae history ----")

    ExpResults.append((hiddenLayers,lossHis,maeHis))
    
    print("-------------------------------------------------")


    

pickle_out = open("hl.pickle","wb")
pickle.dump(ExpResults, pickle_out)
pickle_out.close()

cleanSession()

-------------------------------------------------
Epoch 1/100
12/12 [==============================] - 0s 11ms/step - loss: 1583794685325869056.0000 - mae: 328082368.0000 - val_loss: 1595458574053665669120.0000 - val_mae: 39163842560.0000
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 323098329424236652833538550108520448.0000 - mae: 147227355738275840.0000 - val_loss: inf - val_mae: 18374885491276251136.0000
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: inf - mae: 75301947433693732382179328.0000 - val_loss: inf - val_mae: 7314203680131459543010902016.0000
Epoch 4/100
12/12 [==============================] - 0s 3ms/step - loss: inf - mae: 29802695868212980309918623595495424.0000 - val_loss: inf - val_mae: inf
Epoch 5/100
12/12 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/100
12/12 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_

In [9]:
# third experiment

# we do a trail experiment

ExpResults = []

neuronsNumber = [ 5,25,125]

for nn in neuronsNumber:

    print("-------------------------------------------------")
    
    run_logdir = get_run_logdir("nn",nn)
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

    model = build_model(defaultNOL,nn,optimizer,defaultLR)

    history = model.fit(X_train, y_train, epochs=100,validation_split = 0.1, callbacks=[early_stopping_cb,tensorboard_cb])
    history.history['loss'].sort()
    
    lossHis = history.history["loss"][0]
    print(str(lossHis) + " : loss history ----")
    
    history.history['mae'].sort()
    maeHis = history.history['mae'][0]
    print(str(maeHis) + " : mae history ----")

    ExpResults.append((nn,lossHis,maeHis))
    
    print("-------------------------------------------------")


    

pickle_out = open("nn.pickle","wb")
pickle.dump(ExpResults, pickle_out)
pickle_out.close()

cleanSession()

-------------------------------------------------
Epoch 1/100
12/12 [==============================] - 0s 13ms/step - loss: 6524.3066 - mae: 51.6576 - val_loss: 505.4943 - val_mae: 21.5131
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 594.8486 - mae: 22.4853 - val_loss: 505.2645 - val_mae: 21.5078
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 594.6085 - mae: 22.4799 - val_loss: 505.0348 - val_mae: 21.5024
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 594.3683 - mae: 22.4745 - val_loss: 504.8028 - val_mae: 21.4970
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 594.1259 - mae: 22.4692 - val_loss: 504.5702 - val_mae: 21.4916
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 593.8828 - mae: 22.4637 - val_loss: 504.3397 - val_mae: 21.4863
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 593.6420 - mae: 22.4584 - val_loss: 504.1078 - val_ma

In [10]:
# fourth experiment

# we do a trail experiment

ExpResults = []

expOptimizers = { "SGD" : keras.optimizers.SGD(), "Nesterov" :  keras.optimizers.SGD(nesterov = True) ,  "Momentum" : keras.optimizers.SGD(momentum=0.5), "Adam" : keras.optimizers.Adam() }


for optName,opt in expOptimizers.items():

    print("-------------------------------------------------")
    
    run_logdir = get_run_logdir("opt",opt)
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

    model = build_model(defaultNOL,defaultNeuronsOnLayer,opt,defaultLR)

    history = model.fit(X_train, y_train, epochs=100,validation_split = 0.1, callbacks=[early_stopping_cb,tensorboard_cb])
    history.history['loss'].sort()
    
    lossHis = history.history["loss"][0]
    print(str(lossHis) + " : loss history ----")
    
    history.history['mae'].sort()
    maeHis = history.history['mae'][0]
    print(str(maeHis) + " : mae history ----")

    ExpResults.append((optName,lossHis,maeHis))
    
    print("-------------------------------------------------")


    

pickle_out = open("opt.pickle","wb")
pickle.dump(ExpResults, pickle_out)
pickle_out.close()

cleanSession()

-------------------------------------------------
Epoch 1/100
12/12 [==============================] - 0s 13ms/step - loss: 117101.7812 - mae: 123.7935 - val_loss: 108.1399 - val_mae: 8.5337
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 163.8253 - mae: 9.5988 - val_loss: 68.6999 - val_mae: 6.7683
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 113.2871 - mae: 7.8936 - val_loss: 49.6265 - val_mae: 5.5180
Epoch 4/100
12/12 [==============================] - 0s 3ms/step - loss: 97.5250 - mae: 7.2386 - val_loss: 44.5197 - val_mae: 5.2958
Epoch 5/100
12/12 [==============================] - 0s 3ms/step - loss: 91.5391 - mae: 7.0076 - val_loss: 42.7563 - val_mae: 5.2981
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 87.4378 - mae: 7.0010 - val_loss: 42.5495 - val_mae: 5.0830
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 86.9827 - mae: 6.7954 - val_loss: 41.1674 - val_mae: 5.1446
Epoch 8/1

In [11]:
# fourth experiment

# we do a trail experiment

ExpResults = []

momentumValues = [0.1, 0.5, 0.9]

for mom in momentumValues:

    print("-------------------------------------------------")
    
    run_logdir = get_run_logdir("mom",mom)
    tensorboard_cb = keras.callbacks.TensorBoard(run_logdir)

    optimizerMom = keras.optimizers.SGD(momentum = mom)

    
    model = build_model(defaultNOL,defaultNeuronsOnLayer,optimizerMom,defaultLR)
    
    history = model.fit(X_train, y_train, epochs=100,validation_split = 0.1, callbacks=[early_stopping_cb,tensorboard_cb])
    history.history['loss'].sort()
    
    lossHis = history.history["loss"][0]
    print(str(lossHis) + " : loss history ----")
    
    history.history['mae'].sort()
    maeHis = history.history['mae'][0]
    print(str(maeHis) + " : mae history ----")

    ExpResults.append((mom,lossHis,maeHis))
    
    print("-------------------------------------------------")


    

pickle_out = open("mom.pickle","wb")
pickle.dump(ExpResults, pickle_out)
pickle_out.close()

cleanSession()

-------------------------------------------------
Epoch 1/100
12/12 [==============================] - 0s 12ms/step - loss: 117097.6406 - mae: 123.6391 - val_loss: 104.6918 - val_mae: 8.3816
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 151.6685 - mae: 9.1904 - val_loss: 61.9500 - val_mae: 6.4389
Epoch 3/100
12/12 [==============================] - 0s 2ms/step - loss: 105.8230 - mae: 7.6377 - val_loss: 47.2934 - val_mae: 5.3977
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 94.2339 - mae: 7.1140 - val_loss: 43.1925 - val_mae: 5.2576
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 89.9217 - mae: 6.9550 - val_loss: 42.3040 - val_mae: 5.2694
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 86.3587 - mae: 6.9521 - val_loss: 42.5274 - val_mae: 5.0696
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 86.2858 - mae: 6.7735 - val_loss: 40.8609 - val_mae: 5.0941
Epoch 8/1

In [43]:
param_distribs = { "model__n_hidden": [0, 1, 2, 3], "model__n_neurons": [5,25,125],"model__learning_rate": [10**-6,10**-5,10**-4]}
param_distribs["model__optimizer"] =  [ keras.optimizers.SGD(),  keras.optimizers.SGD(nesterov = True) ,  keras.optimizers.SGD(momentum=0.5),  keras.optimizers.Adam() ]
param_distribs["model__momentum"] = [0.1, 0.5, 0.9]

In [44]:
import scikeras
from scikeras.wrappers import KerasRegressor

es = tf.keras.callbacks.EarlyStopping(patience=10, min_delta=1.0, verbose=1)
keras_reg = KerasRegressor(build_model, callbacks=[es])

In [48]:
from sklearn.model_selection import RandomizedSearchCV

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs,n_iter=30, cv=3, verbose=2)

rnd_search_cv.fit(X_train, y_train, epochs=100, validation_split=0.1)



Fitting 3 folds for each of 30 candidates, totalling 90 fits
Epoch 1/100
8/8 [==============================] - 0s 15ms/step - loss: 4228.3408 - mae: 41.6215 - val_loss: 445.6256 - val_mae: 18.1985
Epoch 2/100
8/8 [==============================] - 0s 3ms/step - loss: 473.1085 - mae: 18.5061 - val_loss: 291.4022 - val_mae: 14.4270
Epoch 3/100
8/8 [==============================] - 0s 4ms/step - loss: 334.7518 - mae: 15.4703 - val_loss: 183.5076 - val_mae: 11.8259
Epoch 4/100
8/8 [==============================] - 0s 3ms/step - loss: 256.1659 - mae: 13.3600 - val_loss: 164.1733 - val_mae: 10.9870
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: 208.5113 - mae: 11.7953 - val_loss: 94.9027 - val_mae: 8.5727
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: 186.9752 - mae: 10.9844 - val_loss: 76.2287 - val_mae: 7.6698
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: 149.1158 - mae: 9.5075 - val_loss: 66.0481 - val_mae: 6.9853

/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/metrics/_scorer.py", line 397, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/scikeras/wrappers.py", line 1089, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/scikeras/wrappers.py", line 1688, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/opt/anaconda3/envs/

8/8 [==============================] - 0s 13ms/step - loss: 489663561728.0000 - mae: 280268.5625 - val_loss: 90535503069184.0000 - val_mae: 9349477.0000
Epoch 2/100
8/8 [==============================] - 0s 3ms/step - loss: 1809780798933065269248.0000 - mae: 16791748608.0000 - val_loss: 405697188771664087744512.0000 - val_mae: 626022219776.0000
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: 8229011774092607422094800584704.0000 - mae: 1139725540786176.0000 - val_loss: 1484618470698591396721966937276416.0000 - val_mae: 37888079771467776.0000
Epoch 4/100
8/8 [==============================] - 0s 3ms/step - loss: inf - mae: 72100892805829754880.0000 - val_loss: inf - val_mae: 2225416601168204791808.0000
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: inf - mae: 4131308591396394436132864.0000 - val_loss: inf - val_mae: 139200228361484662212657152.0000
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: inf - mae: 222918516466

/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/metrics/_scorer.py", line 397, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/scikeras/wrappers.py", line 1089, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/scikeras/wrappers.py", line 1688, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/opt/anaconda3/envs/

8/8 [==============================] - 0s 13ms/step - loss: 311732830208.0000 - mae: 228342.2500 - val_loss: 47703996235776.0000 - val_mae: 6760310.5000
Epoch 2/100
8/8 [==============================] - 0s 3ms/step - loss: 616128809665023705088.0000 - mae: 9543495680.0000 - val_loss: 131659501521643998543872.0000 - val_mae: 354875637760.0000
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: 2090265383110823438054695370752.0000 - mae: 603735701061632.0000 - val_loss: 358926284652213090493171775832064.0000 - val_mae: 18612358888816640.0000
Epoch 4/100
8/8 [==============================] - 0s 3ms/step - loss: inf - mae: 31297154281946218496.0000 - val_loss: inf - val_mae: 1023981617687881056256.0000
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: inf - mae: 1904012250794119982481408.0000 - val_loss: inf - val_mae: 55965180240229823892422656.0000
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: inf - mae: 96247712989521025

...

/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/metrics/_scorer.py", line 397, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/scikeras/wrappers.py", line 1089, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/scikeras/wrappers.py", line 1688, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/opt/anaconda3/envs/

8/8 [==============================] - 0s 15ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 2/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 3/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 4/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 5/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 6/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 7/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 8/100
8/8 [==============================] - 0s 3ms/step - loss: nan - mae: nan - val_loss: nan - val_mae: nan
Epoch 9/100
8/8 [==============================] - 0s 3ms/step - loss: nan 

/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:700: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/model_selection/_validation.py", line 687, in _score
    scores = scorer(estimator, X_test, y_test)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/sklearn/metrics/_scorer.py", line 397, in _passthrough_scorer
    return estimator.score(*args, **kwargs)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/scikeras/wrappers.py", line 1089, in score
    return self.scorer(y, y_pred, sample_weight=sample_weight, **score_args)
  File "/opt/anaconda3/envs/tensorflowEnv/lib/python3.6/site-packages/scikeras/wrappers.py", line 1688, in scorer
    return sklearn_r2_score(y_true, y_pred, **kwargs)
  File "/opt/anaconda3/envs/

12/12 [==============================] - 0s 10ms/step - loss: 631.7487 - mae: 21.0473 - val_loss: 282.1006 - val_mae: 15.0685
Epoch 2/100
12/12 [==============================] - 0s 2ms/step - loss: 297.4659 - mae: 14.1535 - val_loss: 166.9690 - val_mae: 11.0536
Epoch 3/100
12/12 [==============================] - 0s 3ms/step - loss: 209.7504 - mae: 11.4012 - val_loss: 109.7118 - val_mae: 8.7071
Epoch 4/100
12/12 [==============================] - 0s 2ms/step - loss: 166.6617 - mae: 9.7230 - val_loss: 80.9416 - val_mae: 7.3247
Epoch 5/100
12/12 [==============================] - 0s 2ms/step - loss: 140.6457 - mae: 8.8718 - val_loss: 60.3580 - val_mae: 6.2856
Epoch 6/100
12/12 [==============================] - 0s 2ms/step - loss: 119.4611 - mae: 8.1387 - val_loss: 51.7250 - val_mae: 5.7813
Epoch 7/100
12/12 [==============================] - 0s 2ms/step - loss: 107.7082 - mae: 7.6334 - val_loss: 44.6361 - val_mae: 5.4706
Epoch 8/100
12/12 [==============================] - 0s 2ms/step 

RandomizedSearchCV(cv=3,
                   estimator=KerasRegressor(callbacks=[<keras.callbacks.EarlyStopping object at 0x7f9eb9bae588>], model=<function build_model at 0x7f9ed4d107b8>),
                   n_iter=30,
                   param_distributions={'model__learning_rate': [1e-06, 1e-05,
                                                                 0.0001],
                                        'model__momentum': [0.1, 0.5, 0.9],
                                        'model__n_hidden': [0, 1, 2, 3],
                                        'model__n_neurons': [5, 25, 125],
                                        'model__optimizer': [<keras.optimizer_v2.gradient_descent.SGD object at 0x7f9ece2983c8>,
                                                             <keras.optimizer_v2.adam.Adam object at 0x7f9ed65cceb8>]},
                   verbose=2)

In [51]:
rnd_search_cv.best_params_

{'model__optimizer': <keras.optimizer_v2.gradient_descent.SGD at 0x7f9ece2983c8>,
 'model__n_neurons': 25,
 'model__n_hidden': 3,
 'model__momentum': 0.5,
 'model__learning_rate': 1e-06}

In [53]:
pickle_out = open("rnd_search.pkl","wb")
pickle.dump(rnd_search_cv.best_params_, pickle_out)
pickle_out.close()

cleanSession()